# Implementing a piston in a sphere in FriCAS
- Thejasvi Beleyur

This notebook shows my attempts at implementing the model in Chp. 12 of Mellow & Beranek. The model relies on getting the coefficients from the $A_{n}$ 1D matrix, which comes from solving: 

$M.a=b$ --> $a=M^{-1}.b$ (eqn. 12.102)


The matrix $M$ is given by (where $m=0,1...N$ and $n=0,1...N$ ):

$M(m+1,n+1) = \frac{I_{mn}+ \left( nh_{n-1}^{(2)}(kR)-(n+1)h_{n+11}^{(2)}(kR) \right)K_{mn}}{2n+1}$ (eqn. 12.103)

$b(m+1) = -jL_{m}$ (eqn. 12.104)

$a(n+1) = A_{n}$ (eqn. 12.105)

So far, I don't expect $K_{mn}$ and $L_{mn}$ to be a problem as they both produce real-valued output. 

## Implementing $I_{mn}$
The $I_{mn}$ term is a rather involved integral that needs to be calculated numerically as it doesn't have an analytical solution. 

$I_{mn} = \int_0^\alpha \left\{\left( nh_{n-1}^{(2)}(kr_{1}) - (n+1)h_{n+1}^{(2)}(kr_{1} \right)P_{n}(\cos\theta)\cos\theta + n(n+1)h_{n}^{(2)}(kr_{1})(P_{n-1}(\cos\theta)-P_{n+1}(\cos\theta))/kr_{1} \right\}P_{m}(\cos\theta)\frac{r_{1}^2}{R^{2}}tan\theta\:d\theta$

Here, $h_{n}^{(2)}(z)$ is the spherical Hankel function of the second kind: 

$h_{n}^{(2)}(z) = j_{n}(z)-iy_{n}(z)$, where $j_{n},y_{n}$ are the spherical Bessel's functions of the first and second kind. 


### The problem: $I_{mn}$ term produces a complex result
The [numerical quadrature](https://fricas.github.io/api/NumericalQuadrature.html?highlight=aromberg) help page documents a series of methods available to numerically integrate functions which produce ```Float``` outputs. When I try integrating the $I_{mn}$ term numerically it fails. Below is my attempt.

In [ ]:
)version
setFormat!(FormatMathJax)$JFriCASSupport

)set output tex on
)set output algebra off

In [ ]:
++  Some of the constant values required for the rest of the functions below 
++ These correspond to the acoustics of the problem. 

freq := 50000;
k := 2*%pi/(330/freq);
ka := 5 ;
a := ka/k;
alpha := %pi/3;
R:= a/sin(alpha);
R := numeric(R)::Float;
r1 := R*cos(alpha)/cos(theta);
NN :=  3 ;--12 + 2*ka/sin(alpha)--


In [ ]:
++ I couldn't find inbuilt spherical bessel's functions, and so had to make custom functions 

-- spherical bessel function-- 
sphBessel: (PositiveInteger, Float) -> Complex Float
sphBessel(n,z) == besselJ(n+0.5,z)*sqrt(%pi/(2*z));
-- spherical neumann function --
sphNeumann: (PositiveInteger, Float) -> Complex Float
sphNeumann(n,z) == besselY(n+0.5,z)*sqrt(%pi/(2*z));

sphHankel2: (Integer, Float) -> Complex(Float)
sphHankel2(n,z) == sphBessel(n,z) - %i*sphNeumann(n,z);

In [ ]:

alternate_hankels: (PositiveInteger, DoubleFloat) -> Complex(DoubleFloat)
alternate_hankels(n,theta) ==   
    r1 := R*cos(alpha)/cos(theta)
    legendreP(n,cos(theta))*cos(theta) + (n+1)*sphHankel2(n+1,k*r1)

alternate_legendres: (PositiveInteger, DoubleFloat) -> Complex(DoubleFloat)
alternate_legendres(n,theta) ==
    r1 := R*cos(alpha)/cos(theta)
    n*(n+1)*sphHankel2(n,k*r1)*(legendreP(n-1,cos(theta)-legendreP(n+1,cos(theta))))/(k*r1)

legendretan: (PositiveInteger, DoubleFloat)-> Complex(DoubleFloat)
legendretan(m,theta) ==
    r1 := R*cos(alpha)/cos(theta)
    legendreP(m,cos(theta))*(r1^2/R^2)*tan(theta)

one_imn_term: (NonNegativeInteger, NonNegativeInteger, DoubleFloat) -> Complex(DoubleFloat)
one_imn_term(m,n,theta) == 
    r1 := R*cos(alpha)/cos(theta)
    (alternate_hankels(n,theta)+alternate_legendres(m,theta))*legendretan(m,theta)

## Integrate $I_{mn}$ for a fixed value of $m,n$
The numerical integration works with only one free variable. 

Let's set m=10, n=20 and integrate over $\theta$

In [ ]:
intg_imn_m10n20(theta:DoubleFloat):Complex(DoubleFloat) == one_imn_term(10,20,theta)

In [ ]:
++ Check that the output for the Imn term works ++ 
intg_imn_m10n20(0.2)

In [ ]:
alpha


In [ ]:
-- Some code from Waldek Hebisch to implement the numerical integration with complex functions 

pl := [0.9681602395_0762608983, 0.8360311073_266357943, 0.6133714327_0059039731, 0.3242534234_0380892904]::List(DoubleFloat);
wl := [0.3302393550_0125976316, 0.0812743883_6157441196_6, 0.1806481606_9485740398, 0.2606106964_029354623, 0.3123470770_4000284007]::List(DoubleFloat);

gauss9(f : DoubleFloat -> Complex(DoubleFloat), x0 : DoubleFloat, h : DoubleFloat) : Complex(DoubleFloat) ==
    h2 := h/(2::DoubleFloat)
    xm := x0 + h2
    s : Complex(DoubleFloat) := f(xm)*first(wl)
    for a in pl for w in rest(wl) repeat
        hh := h2*a
        s := s + w*(f(xm + hh) + f(xm - hh))
    h2*s 

In [ ]:
f1(x : DoubleFloat) : Complex(DoubleFloat) == exp(complex(1, x))

In [ ]:
yufun := exp(complex(1,x))

In [ ]:
complexNumeric(integrate(yufun,x=0..1))

In [ ]:
ad_gauss(f : DoubleFloat -> Complex(DoubleFloat), x0 : DoubleFloat,h : DoubleFloat, eps : DoubleFloat, max_level : Integer) : Complex(DoubleFloat) ==
    val0 := gauss9(f, x0, h)
    h2 := h/(2::DoubleFloat)
    val1 := gauss9(f, x0, h2)
    val2 := gauss9(f, x0 + h2, h2)
    real(abs(val0 - val1 - val2)) < eps => val0
    max_level = 0 =>
        print("max_level too small")
        val0
    eps2 := eps/(2::DoubleFloat)
    ad_gauss(f, x0, h2, eps2, max_level - 1) +
    ad_gauss(f, x0 + h2, h2, eps2, max_level - 1)


In [ ]:
ad_gauss(f1, 0, 1, 1.0e-3, 30) 